**Partie III**

Maintenant qu'on a vu les briques de base, nous allons entraîner un réseau de neurones à couches de convolution (acronyme CNN) sur un problème un peu plus compliqué que la séparation des nuages de points: la reconnaissance des chiffres manuscrits. \\

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

import torch
import torchvision
import torch.nn as nn   
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms

Le jeu de données s'appelle MNIST. Il est dans le dossier partagé. Il comporte des images noir et blanc (1 canal) de 28*28 pixels. \\

Un objet dataset spécifique lui ait consacré dans le module torchvision.datasets. Les cellules suivantes permettent d'afficher quelques images :

In [ ]:
root = '/content/drive/MyDrive/TP_ENM/data'
#root = '/content/drive/Shareddrives/TP_ENM/data'

#transformation (mise au format)
tr=torchvision.transforms.Compose([
   torchvision.transforms.ToTensor(),
   torchvision.transforms.Normalize((0.1307,), (0.3081,))
   ])

#Définition des jeux d'apprentissage:
ds = {'train': torchvision.datasets.MNIST(root, train = True, transform = tr,
                                          download=True),
      'val': torchvision.datasets.MNIST(root, train = False, transform = tr,
                                        download=True)
     }

phases = ['train','val']

#Dataloaders:
bs = 8
loader ={x :  DataLoader(ds[x], batch_size=bs, shuffle=True, num_workers = 4) for x in phases}
#pour accélérer l'apprentissage, on a passé: num_workers = 4 
#(le chargement des données est ainsi parallélisé, pour aller encore plus vite, on utilisera une carte gpu -voir partie IV)

#Tailles (pour le calcul des scores)
dataset_sizes = {x: len(ds[x]) for x in  phases}

#On fige le générateur de nombres aléatoires
random_seed = 1
torch.manual_seed(random_seed)


In [ ]:
x, t = next(iter(loader['train']))

print(x.shape)

fig = plt.figure()
for i in range(8):
  plt.subplot(4,2,i+1)
  plt.tight_layout()
  plt.imshow(x[i,0,:,:], cmap='gray') #, interpolation='none')
  plt.title("Ground Truth: {}".format(t[i]))
  plt.xticks([])
  plt.yticks([])



Nous allons définir maintenant un CNN de faible profondeur (deux couches de convolution).

**Exercice:** Déterminer *N* de façon à ce que le réseau accepte en entrée les images de MNIST.

In [ ]:
N = ...


class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5, padding =2)
        self.conv2 = nn.Conv2d(10, 15, kernel_size=5, padding =2)
        self.fc1 = nn.Linear(N, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, N)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        # ici, le log est appliqué derrière la softmax :
        return F.log_softmax(x, dim=1)

In [ ]:
model = CNN()

#optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum = 0.9)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

# on sélectionne les log vraisemblances pour les vraies classes:
loss_fn =  torch.nn.NLLLoss() 

La boucle d'apprentissage est doublée: on ne met à jour les poids que dans la première partie, consacrée à l'entraînement. Pendant la **validation**, on surveille les performances en généralisation sur des images indépendantes :


In [ ]:
import time
t = time.time()

train_losses = []
val_losses = []

train_accs = []
val_accs = []

# boucle d'apprentissage:
for epoch in range(6):
    print('epoch :' + str(epoch))
    
    running_loss_train = 0.
    running_corrects_train = 0.
    running_loss_val = 0.
    running_corrects_val = 0.
    
    # entraînement
    for x, label in loader['train']:
        optimizer.zero_grad()
        output = model(x)
        l = loss_fn(output, label)
        l.backward()     
        optimizer.step()

        #prédictions:
        _, preds = torch.max(output, 1)

        # compteurs
        running_loss_train += l.item() * x.shape[0]
        running_corrects_train += torch.sum(preds == label.data)

    # calcul et stockage des scores d'entraînement

    epoch_loss_train = running_loss_train / dataset_sizes['train']
    epoch_acc_train = running_corrects_train.float() / dataset_sizes['train']  

    #Stockage : à coder

    print('{} Loss: {:.4f} Acc: {:.4f}'.format(
        'train', epoch_loss_train, epoch_acc_train))


    # validation
    model.eval()

    for x, label in loader['val']:
                
        with torch.no_grad():
            output = model(x)
            l = loss_fn(output, label)        
            
        # prédictions:
        _, preds = torch.max(output, 1)

        # compteurs:
        running_loss_val += l.item() * x.shape[0]
        running_corrects_val += torch.sum(preds == label.data)
    
    # calcul et stockage des scores en validation
    epoch_loss_val = running_loss_val / dataset_sizes['val']
    epoch_acc_val = running_corrects_val.float() / dataset_sizes['val']    

    #Stockage : à coder

    print('{} Loss: {:.4f} Acc: {:.4f}'.format(
        'val', epoch_loss_val, epoch_acc_val))

    new_t = time.time()
    print('time ' +str(round(new_t- t)))
    t = new_t


**Exercice**: A chaque époque, stocker la justesse (*accuracy*) et la valeur de la fonction de coût dans les listes train_losses, val_losses,
train_accs et val_accs. \\
Tracer les **courbes d'apprentissage** sur six époques.

**Exercice:** Reprendre le perceptron à deux couches (*fc1* et *fc2*) du réseau précédent et le modifier pour prendre directement les imagettes de MNIST en entrée. 
Comparer le perceptron seul au CNN en termes de taille (nombre de poids) et de performances.